# Import Commands

In [1]:
#%qtconsole
#%matplotlib inline

import matplotlib.pyplot as plt
from functools import partial
import numpy as np
import time 
import qcodes as qc
import os
import openpyxl

from qcodes import Station, load_or_create_experiment, initialise_database, Measurement, load_by_run_spec, load_by_guid, initialise_or_create_database_at 
from qcodes.utils.dataset.doNd import do1d, do2d, plot

from qcodes_contrib_drivers.drivers.Tektronix.Keithley_2700 import Keithley_2700
from qcodes.instrument_drivers.tektronix.Keithley_2600_channels import Keithley_2600
from qcodes_contrib_drivers.drivers.Oxford.IPS120 import OxfordInstruments_IPS120
from qcodes.instrument.visa import VisaInstrument
from qcodes_contrib_drivers.drivers.StanfordResearchSystems.SIM928 import SIM928
from qcodes.instrument_drivers.stanford_research.SR830 import SR830
from qcodes.instrument_drivers.stanford_research.SR860 import SR860
from qcodes.dataset.plotting import plot_dataset

# Initialize Instruments

In [2]:
#Magnet Controller
#mag120 = OxfordInstruments_IPS120('ips','TCPIP0::169.254.255.2::7020::SOCKET',23)
#mag120 = OxfordInstruments_IPS120('ips','GPIB0::23::INSTR')
#Lockins(SR860 and SR830)
rxx2=SR860('Bob', 'GPIB1::4::INSTR')
#rxy=SR830('Dilbert','GPIB1::6::INSTR' )
#SIM900 Controller(DC Voltage source)
sim=SIM928('zim','GPIB1::15::INSTR')

Connected to: Stanford_Research_Systems SR860 (serial:004479, firmware:V1.51) in 0.82s
Connected to: Stanford_Research_Systems SIM900 (serial:s/n130213, firmware:ver3.6) in 1.53s


# Set Parameters for Instruments

In [3]:
#Parameters for SIM Controller
#sim.volt_2()
#sim.volt_2.step=0.001
#sim.volt_2.inter_delay=0.2
waitTime = 1
amp = 1
step_size = 0.001
V1_start = 0.0
V1_end = -0.1
V1_SIMPORT = 6
path = "C:\\Users\\Admin\\Desktop\\MattMann\\8-3-22.1MM11C5\\nu=2_QPC_Conductance\\QPC7&3 With QPC17&18 Off.xlsx"

# Sweep Up to Starting Voltage

In [4]:
initial = sim.get_voltage(V1_SIMPORT)
print(initial)
if V1_start>initial:
    for i in np.arange(initial, V1_start, step_size):
        sim.set_voltage(V1_SIMPORT, i)
        time.sleep(waitTime)
else:
    for i in np.arange(initial, V1_start, -1*step_size):
        sim.set_voltage(V1_SIMPORT, i)
        time.sleep(waitTime)

0.4


# Set Up Database for Data Collection

In [7]:
initialise_or_create_database_at("C:\\Users\\Admin\\Desktop\\MattMann\\8-3-22.1MM11C5\\Capacitance Correction\\4 Terminal QPC7&3 With Plungers Closed.db")

# Run Measurement

In [21]:

#exp = load_or_create_experiment('Capacitance Correction with QPC7&3 With QPC18')
#meas = Measurement()
#meas.register_parameter(rxx2.X)

workbook = openpyxl.Workbook()
sheet = workbook.active 

c1 = sheet.cell(row = 1, column = 1) 
c1.value = "Vg1"
c2 = sheet.cell(row = 1, column = 2) 
c2.value = "Vg2"
c3 = sheet.cell(row = 1, column = 3) 
c3.value = "Current"
c4 = sheet.cell(row = 1, column = 4) 
c4.value = "Phase"
curr = 2


#Sweep Down
print("Sweep Down")
for j in np.arange(V1_start, V1_end, -1*step_size):
    temp1 = sheet.cell(row = curr, column = 1)
    temp2 = sheet.cell(row = curr, column = 2)
    temp3 = sheet.cell(row = curr, column = 3)
    temp4 = sheet.cell(row = curr, column = 4)
    sim.set_voltage(V1_SIMPORT, j)
    time.sleep(waitTime)
    temp1.value = i
    temp2.value = j
    (X, P) = rxx2.get_values('X','P')
    temp3.value = X/amp
    temp4.value = P
    print(X)
    curr = curr + 1
    
workbook.save(filename=path)

Sweep Down
-0.05706481263
-0.057071655989
-0.057044576854
-0.057024724782
-0.057104367763
-0.057109832764
-0.057063069195
-0.057030495256
-0.057055931538
-0.05710215494
-0.057038921863
-0.057052820921
-0.057070504874
-0.057078421116
-0.057052638382
-0.057060491294
-0.057041484863
-0.05706454441
-0.057053484023
-0.057037334889
-0.05706281215
-0.057071812451
-0.057064063847
-0.057044733316
-0.057081371546
-0.057066190988
-0.057051796466
-0.057006593794
-0.057031232864
-0.057063590735
-0.057047087699
-0.057062007487
-0.05707764253
-0.057048182935
-0.057041626424
-0.057083405554
-0.057104766369
-0.057104587555
-0.057059753686
-0.057039625943
-0.057008236647
-0.056975930929
-0.056982774287
-0.056955039501
-0.056883823127
-0.056963291019
-0.056970711797
-0.056959863752
-0.057041522115
-0.057045873255
-0.05705454573
-0.057015683502
-0.057047046721
-0.057029373944
-0.057044673711
-0.056951519102
-0.056976087391
-0.05697228387
-0.056917816401
-0.056838322431
-0.05676939711
-0.056768804789
-0.05

-1.2032814993e-06
Sweep Up
-3.1370538522e-07
-3.8438466277e-07
-1.705877537e-07
2.1374671633e-06
4.9103545052e-07
-3.3236830177e-07
-3.691092445e-07
-1.2083361298e-06
4.6478086801e-07
-5.3518277809e-07
-1.3840198108e-06
-2.3251066068e-06
-8.9933718073e-07
4.875202535e-07
1.7648257256e-07
-1.5438625951e-06
1.8514180056e-06
-1.2839734609e-06
-1.5913319658e-06
-1.3981879192e-06
-1.0281675031e-06
-8.9433859785e-07
-1.1984103594e-06
3.4577567476e-06
1.0312172662e-06
-5.8678222103e-07
-1.3857123804e-06
1.9967001208e-06
8.1909684013e-07
-7.9125413777e-07
-7.0689856102e-07
-2.7644716738e-06
4.2153251911e-07
-6.3685837404e-07
-6.5750003841e-07
-6.2742651608e-07
-5.2493118119e-07
-1.4607074945e-06
-8.3616214397e-07
1.5588042288e-06
1.9411709218e-06
4.6504712259e-07
-3.5485445551e-06
-1.1511700677e-06
1.6018478846e-06
-1.2423909084e-06
3.6765570144e-07
-1.5316687723e-06
6.3537345341e-07
-8.2285924918e-07
-1.3773150158e-06
3.144412176e-06
1.1428822972e-06
-8.0290510596e-07
-6.431600923e-07
1.70015

-0.057012625039
-0.05706044659
-0.057041287422


In [7]:
rxx2.get_values('X','P')

(-0.057344324887, -178.79888916)

# Set Values Back to Zero

In [17]:

initial = sim.get_voltage(V1_SIMPORT)
if 0>initial:
    for i in np.arange(initial, 0, step_size):
        sim.set_voltage(V1_SIMPORT, i)
        time.sleep(waitTime)
else:
    for i in np.arange(initial, 0, -1*step_size):
        sim.set_voltage(V1_SIMPORT, i)
        time.sleep(waitTime)

KeyboardInterrupt: 